# LTV

In [35]:
from mantle import mantle
from env import *
from json import dumps
from ipynb.fs.defs.common_last_synced_height import last_synced_height


def query_ltv(wallet_addr: str):
    block_height = last_synced_height()

    return mantle(
        wasm_query={
            'overseer_whitelist': {
                'contractAddress': moneymarket_overseer_addr,
                'query': {
                    'whitelist': {}
                }
            },
            'oracle_prices': {
                'contractAddress': moneymarket_oracle_addr,
                'query': {
                    'prices': {}
                }
            },
            'market_borrower_info': {
                'contractAddress': moneymarket_market_addr,
                'query': {
                    'borrower_info': {
                        'borrower': wallet_addr,
                        'block_height': block_height
                    }
                }
            },
            'overseer_collaterals': {
                'contractAddress': moneymarket_overseer_addr,
                'query': {
                    'collaterals': {
                        'borrower': wallet_addr
                    }
                }
            },
            'overseer_borrow_limit': {
                'contractAddress': moneymarket_overseer_addr,
                'query': {
                    'borrow_limit': {
                        'borrower': wallet_addr,
                        'block_height': block_height
                    }
                }
            }
        }
    )


print(dumps(query_ltv('terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9'), indent=2))

{
  "market_borrower_info": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "interest_index": "1.065820927764831231",
    "loan_amount": "68797125",
    "pending_rewards": "1043335.412796347017660246",
    "reward_index": "708.22385288997003553"
  },
  "oracle_prices": {
    "Ok": {
      "prices": [
        {
          "asset": "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
          "last_updated_time": 1629042734,
          "price": "3157.70996638"
        },
        {
          "asset": "terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x",
          "last_updated_time": 1629042723,
          "price": "16.994835635919666177"
        }
      ]
    }
  },
  "overseer_borrow_limit": {
    "borrow_limit": "109206634",
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9"
  },
  "overseer_collaterals": {
    "borrower": "terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9",
    "collaterals": [
      [
        "terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l",
        "11

In [36]:
from numpy import array, sum

result = query_ltv('terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9')

if result['oracle_prices']['Ok'] is not None:
    oracle_prices = result['oracle_prices']['Ok']['prices']
else:
    oracle_prices = result['oracle_prices']['prices']

# create vectors
# oracle and whitelist and collateral arrangements are not in the same order
# So we need to sort them by token addresses
token_addrs = list(
    map(
        lambda item: item['asset'],
        oracle_prices
    )
)

print('token_addrs', token_addrs)

locked_amounts = []

user_collaterals = result['overseer_collaterals']['collaterals']
for token_addr in token_addrs:
    locked_amount = next((item for item in user_collaterals if item[0] == token_addr), None)
    locked_amounts.append(float(locked_amount[1]))

# vectors
locked_amounts = array(locked_amounts)
prices = array(
    list(
        map(
            lambda item: float(item['price']),
            oracle_prices
        )
    )
)

print('locked_amounts (in Tokens)', locked_amounts)
print('prices (in UST)', prices)

# locked_values = [ ...locked_amounts = token amounts ] * [ ...prices = token ust prices ]
locked_values = locked_amounts * prices

print('locked_values (in UST)', locked_values / 1000000)

total_locked_value = sum(locked_values)

print('total_locked_value (in UST)', total_locked_value / 1000000)

user_ltv = float(result['market_borrower_info']['loan_amount']) / total_locked_value

print('user_ltv (%)', user_ltv * 100)

token_addrs ['terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x']
locked_amounts (in Tokens) [  11350. 8600910.]
prices (in UST) [3157.70996638   16.99483564]
locked_values (in UST) [ 35.84000812 146.17105177]
total_locked_value (in UST) 182.01105988775083
user_ltv (%) 37.79832118027789


In [37]:
max_ltvs = array(
    list(
        map(
            lambda item: float(item['max_ltv']),
            result['overseer_whitelist']['elems']
        )
    )
)

print('max_ltvs (%)', max_ltvs * 100)

max_ltv = sum(max_ltvs) / 2

print('max_ltv (%)', max_ltv * 100)

safe_ratio = 0.75 # constant
safe_ltv = max_ltv * safe_ratio

print('safe_ltv (%)', safe_ltv * 100)

max_ltvs (%) [60. 60.]
max_ltv (%) 60.0
safe_ltv (%) 44.99999999999999


In [38]:
print('user_ltv (%)', user_ltv * 100)
print('safe_ltv (%)', safe_ltv * 100)
print('max_ltv (%)', max_ltv * 100)

user_ltv (%) 37.79832118027789
safe_ltv (%) 44.99999999999999
max_ltv (%) 60.0
